OpenVINO 준비

In [1]:
%pip install -q "openvino>=2023.1.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 21.1 MB/s eta 0:00:00


In [2]:
!pip install openvino-dev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.5 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system 

keras .tf 모델을 Intel OpenVINO IR 파일(xml, bin)로 한다.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import openvino
import openvino as ov
from openvino.tools.mo import convert_model

BASE_PATH = './drive/MyDrive/Colab Notebooks/contest/data/'
ov_model = convert_model(BASE_PATH+'bin/MySelf.tf')
ov.save_model(ov_model, BASE_PATH+'bin/mobilenetv2_class4.xml')

### 테스트 파일 목록으로 읽어와서 답안 출력하기.



In [5]:
# --------------------------- Step 1. Initialize OpenVINO Runtime Core ------------------------------------------------
core = ov.Core()

In [6]:
# --------------------------- Step 2. Read a model --------------------------------------------------------------------
# (.xml and .bin files) or (.onnx file)
model = core.read_model(BASE_PATH+'bin/mobilenetv2_class4.xml')

if len(model.inputs) != 1: print('Sample supports only single input topologies')
if len(model.outputs) != 1: print('Sample supports only single output topologies')

In [7]:
import os

directory_path = "./drive/MyDrive/Colab Notebooks/contest/data/valid"
#directory_path = "./drive/MyDrive/Colab Notebooks/data/test"

# Get a list of all files in the directory
file_list = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# Print the list of files
print("List of files in the directory:")
for file in file_list:
    print(file)

List of files in the directory:
17yo_A.jpg
0_0_02.jpg
0_v_9B.jpg
0d2E8g.jpg
0S_ZY_.jpg
0_zDfn.jpg
126Ek_.jpg
1G0_Z9.jpg
0m_wUL.jpg
1976ku.jpg
0tTcFg.jpg
046w29.jpg
1f4WOP.jpg
1gmS4_.jpg
02L38o.jpg
0N8A9_.jpg
0e_sET.jpg
19_88_.jpg
0qUaOB.jpg
0v40Gb.jpg
13TXK9.jpg
0SH0ge.jpg
02x_DZ.jpg
03T7NT.jpg
17_4M8.jpg
0e_6x3.jpg
0PJb6K.jpg
1eo1_W.jpg
0i1vdc.jpg
0KHL57.jpg
0ScHiO.jpg
1evyt_.jpg
137492.jpg
123_pA.jpg
0576Qr.jpg
0__4_v.jpg
190E3d.jpg
0B02VC.jpg
0O2x31.jpg
0WS5Sa.jpg
0ojCP8.jpg
079hEu.jpg
31mqg1.jpg
1Tai2j.jpg
2lF7F3.jpg
3GVu__.jpg
3BEJ4_.jpg
1ZG6nY.jpg
2uOJ1A.jpg
2_1Sfm.jpg
3G3AeY.jpg
2vO1ZL.jpg
2ZRF6q.jpg
2_h_Ij.jpg
1ss5_T.jpg
2g5W__.jpg
2_L2iP.jpg
30vwgn.jpg
2MgE_G.jpg
2gmZnB.jpg
31dVZK.jpg
2gA7M4.jpg
281FGu.jpg
3Bev14.jpg
3990uB.jpg
1RasnU.jpg
1ZJ90p.jpg
2XM4f3.jpg
25thI8.jpg
1lz3z7.jpg
26Oq_r.jpg
2l6dPa.jpg
1kZL_P.jpg
1iT4_t.jpg
1o6p9_.jpg
2H910Q.jpg
370sa_.jpg
2P2j1U.jpg
2pQp_R.jpg
2W0n0t.jpg
2eLPY4.jpg
23a5__.jpg
2J8i44.jpg
2HLn7H.jpg
243Mk_.jpg
2_1a2F.jpg
2K36D6.jpg
2q6F_0.jpg


In [8]:
import pickle

with open(BASE_PATH+'bin/class4.pickle', 'rb') as f:
    labels = pickle.load(f)

파일 추론하기

In [9]:
import pandas as pd
import cv2
import numpy as np

# --------------------------- Step 4. Loading model to the device -----------------------------------------------------
print('Loading the model to the plugin')
compiled_model = core.compile_model(model, 'CPU')
img_height = 224

result = []

res = open(BASE_PATH+"result.txt", "w")

print('Starting inference in synchronous mode')
cnt = 0;

for file in file_list:
    # --------------------------- Step 5. Set up input --------------------------------------------------------------------
    # Read input image
    image_path = directory_path + "/" + file
    org_image = cv2.imread(image_path)
    image = cv2.resize(org_image, (img_height,img_height), fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)
    #img = cv2.resize(img, (224,224), fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.

    # Add N dimension
    nchw_tensor = np.expand_dims(image, 0)

    # NHWC -> NCHW
    input_tensor = np.transpose(nchw_tensor, (0,1,2,3))#(0,3,1,2))

    # --------------------------- Step 6. Create infer request and do inference synchronously -----------------------------
    results = compiled_model.infer_new_request({0:input_tensor})

    # --------------------------- Step 7. Process output ------------------------------------------------------------------
    predictions = next(iter(results.values()))
    output_node_name = next(iter(results.keys())) #
    #for (k,v) in results.items():
    #  print(k,'=',v)

    #print("values: ",end=' ')
    #print(results.values())

    # Change a shape of a numpy.ndarray with results to get another one with one dimension
    probs = predictions.reshape(-1)
    #print(outs)
    #print("probs: ",end=' ')
    #print(probs)

    # Get an array of 4 class IDs in descending order of probability
    top_4 = np.argsort(probs)[-4:][::-1]
    print(top_4)

    #res = res[output_node_name]
    #idx = np.argsort(res[0])[-1]
    #prob = res[0][idx]*100

    header = 'class_id            probability'
    print(f'Image path: {image_path}')
    print('Top 4 results: ')
    print(header)
    print('-' * len(header))

    for class_id in top_4:
        probability_indent = ' ' * (len('class_id           ') - len(labels[class_id]) + 1)
        print(f'{labels[class_id]}{probability_indent}{probs[class_id]*100:.7f}')

    #print(labels[top_4[0]], probs[top_4[0]]*100)
    #print(labels[top_4[1]], probs[top_4[1]]*100)
    #result.append(top_4[0])
    #res.write(file+':'+str(labels[top_4[0]])+'\n')
    res.write(file+':top1 = ('+labels[top_4[0]]+')'+str(probs[top_4[0]]*100)+', top2 = ('+labels[top_4[1]]+')'+str(probs[top_4[1]]*100)+'\n')

#res.write(str(result)+'\n')
res.close()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
-------------------------------
Pneumonia           100.0000000
NoPneumonia         0.0000001
Osteoarthritis      0.0000000
NoOsteoarthritis    0.0000000
[3 1 2 0]
Image path: ./drive/MyDrive/Colab Notebooks/contest/data/valid/G9S3ua.jpg
Top 4 results: 
class_id            probability
-------------------------------
Pneumonia           99.9999642
NoPneumonia         0.0000302
Osteoarthritis      0.0000000
NoOsteoarthritis    0.0000000
[0 2 3 1]
Image path: ./drive/MyDrive/Colab Notebooks/contest/data/valid/gEFPcg.jpg
Top 4 results: 
class_id            probability
-------------------------------
NoOsteoarthritis    99.0927875
Osteoarthritis      0.9043941
Pneumonia           0.0019659
NoPneumonia         0.0008471
[3 1 0 2]
Image path: ./drive/MyDrive/Colab Notebooks/contest/data/valid/E4eM4j.jpg
Top 4 results: 
class_id            probability
-------------------------------
Pneumonia           98.4344125
NoPneumonia         1.5655955
NoOsteoarthriti